LAB 4:- SQLITE, SQL ALCHEMY AND NoSQL

OBJECTIVES:-
- To understand how relational databases (SQLite) and object-relational mapping (SQLAlchemy) differ from NoSQL databases (MongoDB with PyMongo).
- To perform basic CRUD operations (Create, Read, Update, Delete) using each approach.


Sqlite:
- A lightweight, file-based relational database.
- Uses SQL queries directly (INSERT, UPDATE, DELETE, SELECT).
- Best suited for small to medium applications, prototyping, and local storage.


In [ ]:
import sqlite3

# Connect to (or create) the database
conn = sqlite3.connect('1example.db')
cursor = conn.cursor()

# Create table with all SQLite data types
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS USERS(
        id INTEGER PRIMARY KEY,   -- Whole number, auto-incremented
        name TEXT NOT NULL,       -- Text string
        age INTEGER,              -- Whole number
        salary REAL,              -- Floating-point number
        is_active BOOLEAN,        -- Boolean (0 or 1)
        bio TEXT,                 -- Large text
        join_date DATETIME,       -- Date and time stored as TEXT
        profile_picture BLOB,     -- Binary data
        misc_data NUMERIC,        -- Numeric value, flexible
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP  -- Timestamp
    )
    """
)

# Insert example data
cursor.execute(
    """INSERT INTO users(name,age,salary,is_active,bio,join_date,profile_picture,misc_data)
       VALUES(?,?,?,?,?,?,?,?)""",
    ("Alice", 30, 55000.50, 1, "Software engineer", "2026-01-05 05:00:00", None, 123.45)
)

# Commit changes
conn.commit()

# Query the table
cursor.execute("SELECT * FROM users")
rows = cursor.fetchall()
for row in rows:
    print(row)

# Close connection after all operations only.
# conn.close()


(1, 'Alice', 30, 55000.5, 1, 'Software engineer', '2026-01-05 05:00:00', None, 123.45, '2026-01-27 11:45:36')
(2, 'Alice', 30, 55000.5, 1, 'Software engineer', '2026-01-05 05:00:00', None, 123.45, '2026-01-27 11:53:17')


In [11]:
#Update salary for a user
cursor.execute("UPDATE users SET salary=? WHERE name=?",(70000.00,"Alice"))
print("\nAfter updating Alice's salary:")
cursor.execute("SELECT * FROM users WHERE name='Alice'")
print(cursor.fetchone())


After updating Alice's salary:
(1, 'Alice', 30, 70000.0, 1, 'Software engineer', '2026-01-05 05:00:00', None, 123.45, '2026-01-27 11:45:36')


In [12]:
#Delete a user
cursor.execute("DELETE FROM users WHERE name=?",("Bob",))
print("\nAfter deleting Bob:")
cursor.execute("SELECT * FROM users")
row=cursor.fetchall()
for row in rows:
    print(row)


After deleting Bob:
(1, 'Alice', 30, 55000.5, 1, 'Software engineer', '2026-01-05 05:00:00', None, 123.45, '2026-01-27 11:45:36')
(2, 'Alice', 30, 55000.5, 1, 'Software engineer', '2026-01-05 05:00:00', None, 123.45, '2026-01-27 11:53:17')


SQL ALCHEMY:-
- An ORM (Object Relational Mapper) for Python.
- Maps Python classes to database tables.
- Allows developers to interact with databases using Python objects instead of raw SQL.
- Provides abstraction, cleaner code, and easier schema management.


In [ ]:
from sqlalchemy import create_engine,Column,Integer,String
from sqlalchemy.orm import declarative_base,sessionmaker
#Setup the connection
engine=create_engine('sqlite:///data.db')
Base=declarative_base()
#Define a table.
class User(Base):
    __tablename__='users'
    id=Column(Integer,primary_key=True)
    name=Column(String)
#Create table in db.
Base.metadata.create_all(engine)
#Interact with data
Session=sessionmaker(bind=engine)
session=Session()

new_user=User(name="BE Computer")
session.add(new_user)
session.commit()


INSERTION OF MULTIPLE ROWS:-

In [32]:
from sqlalchemy import create_engine,Column,Integer,String
from sqlalchemy.orm import declarative_base,sessionmaker
#Setup the connection
engine=create_engine('sqlite:///data2.db')
Base=declarative_base()
#Define a table.
class User(Base):
    __tablename__='users'
    id=Column(Integer,primary_key=True)
    name=Column(String)
    email=Column(String)
    course=Column(String)
#Create table in db.
Base.metadata.create_all(engine)
#Interact with data
Session=sessionmaker(bind=engine)
session=Session()
#Multiple rows
users=[
     User(name="BE Computer",email="be@ncit.edu",course="Engineering"),
     User(name="BSc CSIT",email="csit@ncit.edu",course="Science"),
     User(name="BIT",email="bit@ncit.edu",course="IT"),
]
session.add_all(users)
session.commit()


INSERTION OF SINGLE ROW WITH MULTIPLE COLUMNS:-

In [33]:
user=User(
    name="BE Software",
    email="software@ncit.edu",
    course="Engineering"
)

FETCH MULTIPLE ROWS AND COLUMNS:-

In [34]:
all_users=session.query(User).all()
for user in all_users:
    print(user.id,user.name,user.email,user.course)

1 BE Computer be@ncit.edu Engineering
2 BSc CSIT csit@ncit.edu Science
3 BIT bit@ncit.edu IT


In [35]:
user=session.query(User).filter_by(name='BIT').first()
print(user.name,user.email)

BIT bit@ncit.edu


UPDATE ONE ROW:-

In [42]:
user=session.query(User).filter_by(name='BIT').first()
user.course="Information Technology"
session.commit()
print(session.query(User).filter_by(name='BIT').first().course)


Information Technology


UPDATE MULTIPLE ROWS:-

In [41]:
session.query(User).filter(User.course=="Engineering")\
    .update({"course":"BE Engineering"})
session.commit()
for u in session.query(User).all():
    print(u.id, u.name, u.course)


1 BE Computer BE Engineering
3 BIT Information Technology


DELETE ONE ROW:-

In [ ]:
user = session.query(User).filter_by(name='BSc CSIT').first()
session.delete(user)
session.commit()

DELETE MULTIPLE ROWS:-


In [52]:
session.query(User).filter(User.course=="IT").delete()
session.commit()


NoSQL:-
- A NoSQL document-oriented database.
- Stores data as JSON-like documents (flexible schema).
- PyMongo is the Python driver to connect and perform operations.
- Useful for applications requiring scalability, schema flexibility, and hierarchical data


In [59]:
from pymongo import MongoClient
#Connect to the server
client=MongoClient('mongodb://localhost:27017/')
#Access db and collection
db=client['my_database']
users=db['users_collection']
#Insert a Python Dictionary
users.insert_one({"id":1,"name":"Alice","skills":["Python","AI"]})
#Query data
result=users.find_one({"name":"Alice"})
print(result)

{'_id': ObjectId('6978b84e6fc3df7faaac035e'), 'id': 1, 'name': 'Alice', 'skills': ['Python', 'AI']}


In [62]:
from pymongo import MongoClient
#Connect to the server
client=MongoClient('mongodb://localhost:27017/')
#Access db and collection
db=client['my_database']
users=db['users_collection']
#Create(Insert)
#Single document
users.insert_one({
    "id":1,
    "name":"Alice",
    "age":30,
    "salary":55000.50,
    "is_active":True,
    "bio":"Software engineer",
    "skills":["Python","AI"],
    "join_date":"2026-01-05"
})
#Multiple documents
users_to_insert=[
    {"id":2,"name":"Bob","age":28,"salary":48000.00,"is_active":False,"bio":"Data analyst",
     "skills":["SQL","Excel"],"join_date":"2026-01-04"},
     
    {"id":3,"name":"Charlie","age":35,"salary":62000.75,"is_active":True,"bio":"Project manager",
     "skills":["Management","Leadership"],"join_date":"2026-01-03"}
]
users.insert_many(users_to_insert)
#Read(Find)
#Find all users
print("All users:")
for user in users.find():
    print(user)
#Find one user
alice=users.find_one({"name":"Alice"})
print("\nFind Alice:")
print(alice)    


All users:
{'_id': ObjectId('6978b84e6fc3df7faaac035e'), 'id': 1, 'name': 'Alice', 'skills': ['Python', 'AI']}
{'_id': ObjectId('6978be1c6fc3df7faaac0360'), 'id': 1, 'name': 'Alice', 'age': 30, 'salary': 55000.5, 'is_active': True, 'bio': 'Software engineer', 'skills': ['Python', 'AI'], 'join_date': '2026-01-05'}
{'_id': ObjectId('6978be1c6fc3df7faaac0361'), 'id': 2, 'name': 'Bob', 'age': 28, 'salary': 48000.0, 'is_active': False, 'bio': 'Data analyst', 'skills': ['SQL', 'Excel'], 'join_date': '2026-01-04'}
{'_id': ObjectId('6978be1c6fc3df7faaac0362'), 'id': 3, 'name': 'Charlie', 'age': 35, 'salary': 62000.75, 'is_active': True, 'bio': 'Project manager', 'skills': ['Management', 'Leadership'], 'join_date': '2026-01-03'}

Find Alice:
{'_id': ObjectId('6978b84e6fc3df7faaac035e'), 'id': 1, 'name': 'Alice', 'skills': ['Python', 'AI']}


In [ ]:
#Update Alice's salary
users.update_one({"name":"Alice"},{"$set":{"salary":70000}})
print("\nAfter updating Alice's salary:")
print(users.find_one({"name":"Alice"}))


After updating Alice's salary:
{'_id': ObjectId('6978b84e6fc3df7faaac035e'), 'id': 1, 'name': 'Alice', 'skills': ['Python', 'AI'], 'salary': 70000}


In [64]:
#Delete Bob
users.delete_one({"namr":"Bob"})
print("\nAfter deleting Bob:")
for user in users.find():
    print(user)


After deleting Bob:
{'_id': ObjectId('6978b84e6fc3df7faaac035e'), 'id': 1, 'name': 'Alice', 'skills': ['Python', 'AI'], 'salary': 70000}
{'_id': ObjectId('6978be1c6fc3df7faaac0360'), 'id': 1, 'name': 'Alice', 'age': 30, 'salary': 55000.5, 'is_active': True, 'bio': 'Software engineer', 'skills': ['Python', 'AI'], 'join_date': '2026-01-05'}
{'_id': ObjectId('6978be1c6fc3df7faaac0361'), 'id': 2, 'name': 'Bob', 'age': 28, 'salary': 48000.0, 'is_active': False, 'bio': 'Data analyst', 'skills': ['SQL', 'Excel'], 'join_date': '2026-01-04'}
{'_id': ObjectId('6978be1c6fc3df7faaac0362'), 'id': 3, 'name': 'Charlie', 'age': 35, 'salary': 62000.75, 'is_active': True, 'bio': 'Project manager', 'skills': ['Management', 'Leadership'], 'join_date': '2026-01-03'}


DISCUSSION AND CONCLUSION:-
Exploring SQLite, SQLAlchemy, and MongoDB (PyMongo) shows how different database approaches fit different needs.
- SQLite is simple and portable, ideal for small projects. It requires direct SQL commands and has rigid schemas, meaning structural changes often need dropping and recreating tables.
- SQLAlchemy provides cleaner, object-oriented interaction with databases. It improves maintainability but doesn’t automatically update schemas, so migrations are needed when models change.
- MongoDB/PyMongo offers schema-less flexibility, storing JSON-like documents. It’s well-suited for scalable applications but requires careful handling to avoid inconsistent data.
Overall, each tool has strengths: SQLite for lightweight setups, SQLAlchemy for structured applications, and MongoDB for flexible, evolving data. The lab demonstrated CRUD operations across all three, highlighting both their practical use and common pitfalls. The choice depends on project requirements — simplicity, abstraction, or flexibility

